In [2]:
import numpy as np
import os as os
from datetime import datetime
from tensorflow.python.ops.control_flow_ops import case
from lcpae import LcpAe
from lcpgenerator import LcpGenerator
import matplotlib.pyplot as plt
from tensorflow import keras
from keras import layers
from keras.models import Model
from keras.models import load_model
import tensorflow as tf
import glob as glob



In [11]:
encoder = load_model('/scratch-shared/david/model_data/CRAE_arch/exp147_epochs_60_D20213703_T153737encoderSegment')
encoder.summary()
encoder.compile(optimizer='Adam', loss='mse')
# predGen = LcpGenerator(inpath='../data/ki-database/exp147', batch_size = 1, input_aug=False)
predArray = []
loadpath = sorted(glob.glob('/scratch-shared/david/data/ki-database/exp147'+'/*'))
loadpath = loadpath[3::4]
for i, inpath in enumerate(loadpath):
    print(i)
    print(inpath)
    npseq = np.load(inpath+'/sequence.npy').astype(float)
    npseq = npseq/65535
    npseq = np.reshape(npseq, (1, npseq.shape[0], npseq.shape[1], npseq.shape[2], npseq.shape[3]))
    print(npseq.shape)
    nppred = encoder.predict(npseq)
    predArray += [nppred]

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      [(None, 25, 1080, 1080, 6 0         
_________________________________________________________________
conv_timedist_layer (TimeDis (None, 25, 1050)          127506    
_________________________________________________________________
gru (GRU)                    (None, 25, 500)           2328000   
_________________________________________________________________
gru_1 (GRU)                  (None, 25, 250)           564000    
_________________________________________________________________
gru_2 (GRU)                  (None, 25, 100)           105600    
Total params: 3,125,106
Trainable params: 3,125,106
Non-trainable params: 0
_________________________________________________________________
0
/scratch-shared/david/data/ki-database/exp147/r17c09
(1, 25, 1080, 1080, 6)
1
/scratch-shared/david/data/ki-datab

In [12]:
crae = load_model('/scratch-shared/david/model_data/CRAE_arch/exp147_epochs_60_D20213703_T153737full')
crae.summary()
predArray = []
loadpath = sorted(glob.glob('/scratch-shared/david/data/ki-database/exp147'+'/*'))
print(loadpath)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      [(None, 25, 1080, 1080, 6 0         
_________________________________________________________________
conv_timedist_layer (TimeDis (None, 25, 1050)          127506    
_________________________________________________________________
gru (GRU)                    (None, 25, 500)           2328000   
_________________________________________________________________
gru_1 (GRU)                  (None, 25, 250)           564000    
_________________________________________________________________
gru_2 (GRU)                  (None, 25, 100)           105600    
Total params: 3,125,106
Trainable params: 3,125,106
Non-trainable params: 0
_________________________________________________________________
['/scratch-shared/david/data/ki-database/exp147/r17c03', '/scratch-shared/david/data/ki-database/exp147/r17c05', '/